# Aprendizaje por el gradiente

En esta clase estudiaremos las bases del aprendizaje por el gradiente, técnica muy usada en las redes neuronales artificiales. El objetivo principal de esta notebook es ganar las intuiciones de cómo funcionan estas técnicas.  Para esto trabajaremos con conjuntos de datos simples y modelos simples como regresiones lineales y logísticas.

In [ ]:
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import random
from sklearn.datasets import load_breast_cancer
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.manifold import TSNE
from sklearn.utils import shuffle
from tqdm.notebook import tqdm

'''Esta función dibuja bonita la matríz de confunsión.
'''
def show_confusion_matrix(cm, labels):
    fig = plt.figure()
    ax = fig.add_subplot(111)
    cax = ax.matshow(cm)
    plt.title('Matriz de confusión')
    fig.colorbar(cax)
    ax.set_xticklabels([''] + labels)
    ax.set_yticklabels([''] + labels)
    plt.xlabel('Predicho')
    plt.ylabel('Verdadero')
    for i, row in zip(range(len(cm)), cm):
        for j, val in zip(range(len(row)), row):
            ax.text(i, j, str(val), va='center', ha='center').set_backgroundcolor('white')
    plt.show()

    
random.seed(42)
np.random.seed(42)
mpl.rcParams['figure.figsize'] = [12.0, 8.0]
print(mpl.rcParams['figure.figsize'])

## Regresión Lineal
La regresión lineal es la aplicación de un modelo lineal entre una variable dependiente ($y$) y una o más variables dependientes ($X$).

$$\hat{y}=x_0w_0+x_1w_1+...+x_iw_i+b$$

donde, considerando un error $\varepsilon$:

$$y = \hat{y}+\varepsilon$$

Siendo el caso de una variable:

$$\hat{y}=xw+b$$

A continuación, se presenta un ejemplo basado en datos generados. Para este ejemplo se generarán 50 puntos con la siguiente distribución:

$$y=3*x+(rand-0.5)$$


In [ ]:
def gen_random_data(mult):
    _x = np.linspace(-1, 1, 100)
    _error = (np.random.rand(*_x.shape) - .5)
    _y = _x * mult + _error
    return _x, _y


x, y = gen_random_data(3)
plt.plot(x, y, 'ro')
plt.show()
print('x: {}'.format(x))
print('y: {}'.format(y))

In [ ]:
def lineal(x, w, b):
    return x*w+b

plt.plot(x, y, 'ro', x, lineal(x, 3, 0))
plt.show()

## Objetivo
Considerando la varaible independiente $x$ y la variable dependiente $y$, el objetivo de un regresión lineal es encontrar $w$ y $b$ tal que dada una función de error $E(y, \hat{y})$ sea mínimo. Es decir:

$$\underset{w,b}{arg\,min}=E(y,xw+b)$$

## Función de error
Una función de error utilizada para este tipo de problemas es el error medio cuadrático (_mean squared error_), que se define como:

$$MSE(y,\hat{y})=\frac{1}{N}\sum(y-\hat{y})^{2}$$

In [ ]:
def mse(y_true, y_pred):
    return np.mean((y_true-y_pred)**2) 

print('El MSE es {}'.format(mse(y, lineal(x, 3, 0))))

Para encontrar los parámetros que cumplen con el objetivo de $\underset{w,b}{arg\,min}=E(y,xw+b)$ podríamos explorar extensivamente posibles valores de los parámetros.

In [ ]:
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm

fig = plt.figure()
ax = fig.gca(projection='3d')

# Construyendo datos
w = np.arange(1, 5, 0.1)
b = np.arange(-1, 1, 0.01)
w, b = np.meshgrid(w, b)
e = np.empty_like(w)
for i in range(w.shape[0]):
    for j in range(w.shape[1]):
        e[i, j] = mse(y, lineal(x, w[i, j], b[i, j]))


ax.plot_surface(w, b, e, cmap=cm.coolwarm,
                       linewidth=0, antialiased=False)

ax.set_xlabel('Valores de w')
ax.set_ylabel('Valores de b')
ax.set_zlabel('Función de perdida')
plt.show()

Si bien esta técnica podría servir para encontrar los parámetros no es escalable. Es decir, si hay más parámetros no se podría encontrar los mejores valores en un tiempo razonable. Es importante notar que muchas de las redes neuronales modernas tienen miles o millones de parámetros.

## Optimización de parámetros
A continuación, analizaremos el problema de optimización de parámetros. Para simplificar el problema, solo trabajaremos con el parámetro $w$ y fijaremos $b=0$. Podemos hacer esto, porque para el conjunto de datos artificial, conocemos la parametrización optima.


In [ ]:
def exp_error(y, x, ws):
    def single_error(w):
        return mse(y, lineal(x, w, 0))
    _s = np.vectorize(single_error)
    return _s(ws)

ws = np.linspace(1, 5, 51)
plt.plot(ws, exp_error(y, x, ws))
plt.show()

## Pendiente del error
Dado que la función de error tiene un solo mínimo, se podrían tomar 2 valores cercanos de manera de conocer en qué dirección es conveniente explorar. La función lineal en realidad es una función que depende no solo de los datos $x$, sino que también del parámetro a aprender $w$, entonces la notaremos como $h(x,w)$. Para conocer la pendiente de la función de error dado el parámetro a conocer debemos hacer:

$$pendiente_w(w_{1}, w_{0})=\frac{MSE(y,h(x,w_{1}))-MSE(y,h(x,w_{0}))}{w_{1}-w_{0}}$$ 

In [ ]:
errors = exp_error(y, x, ws)
pendiente = (errors[10]-errors[20])/(ws[10]-ws[20])
correccion_ordenada_origen = -pendiente*ws[10] + errors[10]
plt.plot(ws, errors, ws[10:21], lineal(pendiente, ws[10:21], 0)+correccion_ordenada_origen)
plt.show()

Entonces, se podría inicializar $w$ de forma aleatoria e ir actualizando el valor en contra de la pendiente, ya que la pendiente indica la dirección en la que crece el error con respecto a $w$. El problema es determinar cuanto moverse. En una técnica básica es moverse proporcionalmente al la “inclinación” de la pendiente.

$$w_{n+1}=w_n – lr*pendiente_w $$
 


In [ ]:
def pendiente(y_true, x, w, delta=1e-6):
    return (mse(y_true, lineal(x, w + delta, 0))-mse(y_true, lineal(x, w, 0))) / delta

w = 0 #Podría ser cualquier valor
ciclos = 100
lr = 0.1 
errors = []
ws = []
for i in range(ciclos):
    p = pendiente(y, x, w)
    w = w - lr * p
    ws.append(w)
    errors.append(mse(y, lineal(x, w, 0)))
print('Errores a medida que se actualiza el valor de w')
plt.plot(errors)
plt.show()
print('El w final es {}'.format(w))
print(ws)

### ¿Funcionaría si tenemos más parámetros? ¿Cómo?
Podríamos considerar los parámetros independientemente, es decir, para calcular la pendiente de $w$ fijamos $b$ en el valor arbitrario y viceversa. 


In [ ]:
def pendiente(y_true, x, w, b, delta=1e-6):
    pw = (mse(y_true, lineal(x, w + delta, b))-mse(y_true, lineal(x, w, b))) / delta
    pb = (mse(y_true, lineal(x, w, b + delta))-mse(y_true, lineal(x, w, b))) / delta
    return pw, pb

w = 0 #Podría ser cualquier valor
b = 5
ciclos = 100
lr = 0.1 
errors = []
for i in range(ciclos):
    pw, pb = pendiente(y, x, w, b)
    errors.append(mse(y, lineal(x, w, b)))
    w = w - lr * pw
    b = b - lr * pb
    #Actualicé el valor de b
print('Errores a medida que se actualiza el valor de w y b')
plt.plot(errors)
plt.show()
print('El w final es {}'.format(w))
print('El b final es {}'.format(b))

## Gradient Descent

El problema de utilizar la pendiente es que nos agrega muchos cálculos y el hiper-parámetros $\delta$. Pero, como queremos que el $\delta \rightarrow 0$, podríamos tomar el límite con $\delta$ tendiendo a $0$.

### Pendiente de $w$
Desde el punto de vista de la pendiente de $w$, si definimos a $w_{1}=w_{0}+\Delta$ entonces:

$$\lim_{\Delta \rightarrow 0} pendiente_w(w_{0}+\Delta,w_{0})= \lim_{\Delta \rightarrow 0} \frac{MSE(y,h(x,w_{0}+\Delta, b))-MSE(y,h(x,w_{0}, b))}{\Delta} =\frac{dMSE(y,h(x,w,b))}{dw}$$

Esta derivada se puede resolver por regla de la cadena:

$$\frac{dMSE(y,h(x,w,b))}{dw}=\frac{dMSE(Y,h(x,w,b))}{d(h(x,w,b))}.\frac{(h(x,w,b))}{dw}$$

La primer derivada se resuelve, devuelta por regla de la cadena:

$$\frac{dMSE(y,h(x,w,b))}{d(h(x,w,b)}=\frac{d(\frac{1}{N}\sum(y-h(x,w,b))^{2}}{d(h(x,w,b))}=-\frac{2}{N}\sum(y-h(x,w,b))$$

La segunda derivada se resuelve así:

$$\frac{dh(x,w,b)}{dw}=\frac{d(xw+b)}{dw}=x$$

Finalmente, resulta en:

$$\frac{dMSE(y,h(x,w,b))}{dw}=\frac{dMSE(Y,h(x,w,b))}{d(h(x,w,b))}.\frac{(h(x,w,b))}{dw}=\frac{-2\sum(y-(xw+b))*x}{N}$$

### Pendiente $b$
Desde el punto de vista de la pendiente de $b$, si definimos a $b_{1}=b_{0}+\Delta$ entonces:

$$\lim_{\Delta \rightarrow 0} pendiente_b(b_{0}+\Delta,b_{0})= \lim_{\Delta \rightarrow 0} \frac{MSE(y,h(x,w, b_{0}+\Delta))-MSE(y,h(x,w,b_{0}))}{\Delta} =\frac{dMSE(y,h(x,w,b))}{db}$$

Esta derivada se puede resolver por regla de la cadena:

$$\frac{dMSE(y,h(x,w,b))}{db}=\frac{dMSE(Y,h(x,w,b))}{d(h(x,w,b))}.\frac{(h(x,w,b))}{db}$$

La primer derivada ya se resolvió arriba.

La segunda derivada se resuelve así:

$$\frac{dh(x,w,b)}{db}=\frac{d(xw+b)}{db}=1$$

Finalmente, resulta en:

$$\frac{dMSE(y,h(x,w,b))}{db}=\frac{dMSE(Y,h(x,w,b))}{d(h(x,w,b))}.\frac{(h(x,w,b))}{db}=\frac{-2\sum(y-(xw+b))*1}{N}$$

In [ ]:
def gradiente(y_true, x, w, b):
    dp = y_true - lineal(x, w, b)
    gw = -2 * np.mean(dp * x)
    gb = -2 * np.mean(dp)
    return gw, gb
               
def gradient_check(x, y_true, ciclos=100, delta=1e-6, error=1e-5):
    ok = True
    for i in range(ciclos):
        w = random.uniform(-50, 50) #genera un flotante aleatorio
        b = random.uniform(-50, 50) #genera un flotante aleatorio
        pw, pb = pendiente(y_true, x, w, b, delta)
        gw, gb = gradiente(y_true, x, w, b)
        if abs(pw - gw) > error or abs(pb - gb) > error:
            print('Error para w={}, b={}. pw={}, pb={}, gw={},gb={}'.format(w, b, pw, pb, gw, gb))
            print(pw-gw)
            print(pb-gb)
            ok = False
    if ok:
        print('No hubo errores en la prueba')
        
gradient_check(x, y)

In [ ]:
w = random.uniform(-50, 50) #genera un flotante aleatorio
b = random.uniform(-50, 50) #genera un flotante aleatorio
print('Valores iniciales. w={} b={}'.format(w, b))
ciclos = 100
lr = 0.1
errors = []
for i in range(ciclos):
    gw, gb = gradiente(y, x, w, b)
    errors.append(mse(y, lineal(x, w, b)))
    w = w - lr * gw
    b = b - lr * gb
print('Errores a medida que se actualiza el valor de w')
plt.plot(errors)
plt.show()
print('El w final es {}'.format(w))
print('El b final es {}'.format(b))

## Tensorflow

Como esta es la base de las técnicas para entrenar muchos modelos de aprendizaje de máquina, existen librerías que nos asisten a implementar este tipo de algoritmos. Tensorflow es una de estas librerías que puede calcular gradientes de forma automática, permite aceleración utilizando GPU o TPU, y nos brinda diversas abstracciones de más alto nivel que facilitan la implementación de redes neuronales.

A continuación, veremos una implementación del modelo utilizado para analizar el gradient descent utilizando Tensorflow.


In [ ]:
def tf_lineal(x, w, b):
    return x * w + b

def tf_mse(y_true, y_pred):
    return tf.math.reduce_mean((y_true-y_pred)**2) 

def tf_cost(y_true, x, w, b):
    return tf_mse(y_true, tf_lineal(x, w, b))


#Tipos de datos esperados por tf
x = x.astype(np.float32)
y = y.astype(np.float32)

In [ ]:
w = tf.random.uniform(shape=[], minval=-1, maxval=1)
b = tf.random.uniform(shape=[], minval=-1, maxval=1)
ciclos = 100
lr = 0.1 
errors = []
for i in range(ciclos):
    with tf.GradientTape() as g:
        g.watch([w, b])
        loss = tf_cost(y, x, w, b)
        errors.append(loss.numpy())
    gw, gb = g.gradient(loss, [w, b])
    w = w - lr * gw
    b = b = - lr * gb

print('Errores a medida que se actualiza el valor de w')
plt.plot(errors)
plt.show()
print('El w final es {}'.format(w))
print('El b final es {}'.format(b))

## Clasificación con regresión logística.
La regresión logística es un modelo que permite clasificar instancias en 2 clases. Por este motivo, imagen de esta función está en $(0, 1)$.

$$P(y|x)= sigmoid(xw + b)$$

En este contexto, la función seleccionada para hacer esta estimación por excelencia es la sigmoide.

$$sigmoid(z)=\frac{1}{1+e^{-z}}$$

### Función de error
Para calcular el error, se utiliza la entropía cruzada entre el valor esperado y el valor obtenido.
$$CE(y,\hat{y})=\frac{\sum(-y*log(\hat{y})-(1-y)*log(1-\hat{y}))}{N}$$
En este contexto, la entropía curzada se interpreta como la información promedio (en bits) necesaria para determinar el valor de $y$ dado que se conoce el valor de $\hat{y}$.

### Ejemplo
Para el ejemplo de regresión logística se utilizará el conjunto de datos de cancer de pecho provisto. Este conjuntos de datos fue recolectado por investigadores de la Universisda de Wisconsin y provisto por la [UCI](https://archive.ics.uci.edu/ml/datasets/Breast+Cancer+Wisconsin+(Diagnostic)). Para acceder al conjunto de datos, no es necesario descargarlo y convertirlo al formato, ya que en encuentra provisto por el módulo de [_sklearn.datasets_](http://scikit-learn.org/stable/modules/generated/sklearn.datasets.load_breast_cancer.html#sklearn.datasets.load_breast_cancer) de la librería sickit-learn, que es una librería de _machine learning_ que se utilizará durante el curso.

El dataset tiene 569 instancias, con 30 atributos cada una. Las instancias pueden ser clasificadas entre Malignos y Benignos. El dataset está ligeramente desbalanceado, lo que significa que existen más instancias de una clase que de la otra. En partícular, 37,25% de las instancias son Malignas y 62,75% son Benignas. La siguiente tabla resume el conjunto de datos:

| Propiedad | Valor |
| --- | --- |
| Clases | 2 |
| Ejemplos por clase | 212(M-0), 357(B-1) | 
| Total de instancias | 569 |
| Dimensionalidad | 30|

El siguiente código:
1. Levanta los datos divididos en `x` (atributos) e `y` (clase).
1. Divide los datos en entrenamiento y testing.
1. Escala los datos de entrenamiento a valores entre 0 y 1.
1. Aplica las correcciones de escalado al conjunto de testing.


In [ ]:
x, y = load_breast_cancer(True)
x_train = x[:500,:]
y_train = y[:500]
x_test = x[500:,:]
y_test = y[500:]

maxs = np.max(x_train, axis=0)
mins = np.min(x_train, axis=0)
x_train = (x_train - mins) / (maxs - mins)
x_test = (x_test - mins) / (maxs - mins)

x_train = x_train.astype(np.float32)
x_test = x_test.astype(np.float32)
y_train = y_train.astype(np.float32)
y_test = y_test.astype(np.float32)

In [ ]:
ts_rep = TSNE().fit_transform(x_train)
for point, label in zip(ts_rep, y_train):
    rep = 'b*' if label == 1 else 'r*'
    plt.plot([point[0]], [point[1]], rep)
plt.show()

In [ ]:
def logisticRegresion(x, w, b):
    return 1/(1+tf.exp(-(tf.matmul(x, w) + b)))[:,0]

In [ ]:
w = tf.random.uniform(shape=[30, 1]) - 0.5
b = tf.random.uniform(shape=[]) - 0.5

In [ ]:
def crossentropy(yt, yp):
    return tf.math.reduce_mean(-yt*tf.math.log(tf.clip_by_value(yp, 1e-6, 1)) - (1-yt)*tf.math.log(tf.clip_by_value(1-yp, 1e-6, 1)))

In [ ]:
crossentropy(y_train, logisticRegresion(x_train, w, b))

In [ ]:
v = logisticRegresion(x_train, w, b).numpy()
print(np.min(v))
print(np.max(v))

In [ ]:
w = tf.random.uniform(shape=[30, 1], minval=-1, maxval=1)
b = tf.random.uniform(shape=[], minval=-1, maxval=1)
ciclos = 100
lr = 0.1 
errors = []
for i in range(ciclos):
    with tf.GradientTape() as g:
        g.watch([w, b])
        loss = crossentropy(y_train, logisticRegresion(x_train, w, b))
        errors.append(loss.numpy())
    gw, gb = g.gradient(loss, [w, b])
    w = w - lr * gw
    b = b - lr * gb

print('Errores a medida que se actualiza el valor de w')
plt.plot(errors)
plt.show()
print('El w final es {}'.format(w))
print('El b final es {}'.format(b))

In [ ]:
y_pred = logisticRegresion(x_test, w, b).numpy()
show_confusion_matrix(confusion_matrix(y_test, y_pred > 0.5), ['Maligno', 'Benigno'])
print(classification_report(y_test, y_pred > 0.5))

## Stochastic Gradient Descent
Hasta el momento calculamos el gradiente sobre todo el conjunto de datos de entrenamiento. Sin embargo, esto puede no ser posible o eficiente cuando tenemos un conjunto de datos grande. Por esto, se suele dividir el conjunto de datos en mini-batch y entrenar sobre estos mini-batchs. La idea, es que en promedio la agregación de los efectos de la actualización en cada mini-batch nos acerque al mínimo. Obviamente, esto no significa que cada actualización nos acerque al mínimo global.


In [ ]:
w = tf.random.uniform(shape=[30, 1], minval=-1, maxval=1)
b = tf.random.uniform(shape=[], minval=-1, maxval=1)
ciclos = 100
lr = 0.1 
errors = []
errors_minibatch = []
errors_minibatch2 = []
for i in range(ciclos):
    x_s, y_s = shuffle(x_train, y_train)
    for mini_batch in range(0, 500, 50):
        with tf.GradientTape() as g:
            g.watch([w, b])
            loss = crossentropy(y_s[mini_batch:mini_batch+50], logisticRegresion(x_s[mini_batch:mini_batch+50], w, b))
            errors_minibatch2.append(loss.numpy())
            errors_minibatch.append(crossentropy(y_train, logisticRegresion(x_train, w, b)).numpy())
        gw, gb = g.gradient(loss, [w, b])
        w = w - lr * gw
        b = b - lr * gb
    errors.append(crossentropy(y_train, logisticRegresion(x_train, w, b)).numpy())

print('Errores a medida que se actualiza el valor de w')
plt.plot(errors)
plt.show()
plt.plot(errors_minibatch)
plt.show()
plt.plot(errors_minibatch2)
plt.show()
print('El w final es {}'.format(w))
print('El b final es {}'.format(b))

In [ ]:
y_pred = logisticRegresion(x_test, w, b).numpy()
show_confusion_matrix(confusion_matrix(y_test, y_pred > 0.5), ['Maligno', 'Benigno'])
print(classification_report(y_test, y_pred > 0.5))

## Momentum
Para mejorar los resultados una técnica muy utilizada es agregar un momentum. La idea del momentum es que vaya llevando una historia del movimiento entre los mini-batchs, y como en promedio esperamos que nos lleve a un mínimo global el momentum llevaría nuestras actualizaciones más hacia el este mínimo.

$$vel_n=momentum * vel_{n-1} – lr * grad_n$$

$$w_{n+1} = w_{n} + vel_n$$ 


In [ ]:
w = tf.random.uniform(shape=[30, 1], minval=-1, maxval=1)
b = tf.random.uniform(shape=[], minval=-1, maxval=1)
ciclos = 100
lr = 0.1 
momentum = 0.9
errors = []
errors_minibatch = []
errors_minibatch2 = []
vw = tf.zeros(shape=[30, 1])
vb = tf.zeros(shape=[])
for i in range(ciclos):
    x_s, y_s = shuffle(x_train, y_train)
    for mini_batch in range(0, 500, 50):
        with tf.GradientTape() as g:
            g.watch([w, b])
            loss = crossentropy(y_s[mini_batch:mini_batch+50], logisticRegresion(x_s[mini_batch:mini_batch+50], w, b))
            errors_minibatch.append(crossentropy(y_train, logisticRegresion(x_train, w, b)).numpy())
            errors_minibatch2.append(loss.numpy())
        gw, gb = g.gradient(loss, [w, b])
        vw = momentum * vw - lr * gw
        vb = momentum * vb - lr * gb
        w = w + vw
        b = b + vb
    errors.append(crossentropy(y_train, logisticRegresion(x_train, w, b)).numpy())

print('Errores a medida que se actualiza el valor de w')
plt.plot(errors)
plt.show()
plt.plot(errors_minibatch)
plt.show()
plt.plot(errors_minibatch2)
plt.show()
print('El w final es {}'.format(w))
print('El b final es {}'.format(b))

In [ ]:
y_pred = logisticRegresion(x_test, w, b).numpy()
show_confusion_matrix(confusion_matrix(y_test, y_pred > 0.5), ['Maligno', 'Benigno'])
print(classification_report(y_test, y_pred > 0.5))

## Clasificación multiclase
Para la clasificación multiclase una función muy utilizada es el softmax. Esta función toma por entrada un vector y retorna un vectos tal que la suma de todos sus elementos es $1$ y todos los valores están entre $0$ y $1$. Si nuestro problema de clasificación tiene n clases, podemos usar la función softmax y un vector n-dimensiones. Entonces, podemos interpretar la salida de esta función como la distribución de probabilidades de las clases.
$$softmax_i(x) = \frac{e^{x}}{\sum e^{x}} $$

## Categorical Crossentropy
Esta función de error considera el error sobre la categoría real, normalizando el valor de la predicción. Considerando $\hat{y}=(\hat{y}_1, \hat{y}_2, ..., \hat{y}_C)$ in vector de valores asociados a las clases
$$P_\hat{y}=\frac{\hat{y}}{\sum\hat{y}_i}$$
$$CCE(y,P_\hat{y})=-\frac{\sum y * log(P_\hat{y})}{N} $$
Notese que el valor de error se considera solo sobre las clases verdaderas, las otras son afectadas a través de la normalización de la salida $\hat{y}$.

## Problema de OCR de dígitos
Para este trabajos utilizaremos el conjunto de datos conocido como [MNIST](http://yann.lecun.com/exdb/mnist/). Este conjunto de datos ya se encuentra dividido entre entrenamiento y testing. El problema consiste en clasificar imagenes de dígistos escritos a mano al dígito correspondiente.

| Propiedad | Valor |
| --- | --- |
| Clases | 10 |
| Tamaño de las imagenes | 28 X 28 |
| Instancias de entrenemiento | 60.000 |
| Instancias de entrenemiento | 10.000 |
| Valor mínimo de cada pixel | 0 |
| Valor máximo de cada pixel | 255 |

A continuación se carga el dataset y se dibujan los primeros 100 ejemplos del conjunto de entrenamiento.



In [ ]:
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical
(x_train, y_train), (x_test, y_test) = mnist.load_data()

print('100 primeros elementos del conjunto de entrenaimento')
f = plt.figure(111)
for i in range(10):
    for j in range(10):
        ax = f.add_subplot(10, 10, i + j*10 + 1)
        ax.set_xticklabels('')
        ax.set_yticklabels('')
        ax.imshow(x_train[i + j*10, :, :], cmap='gray')
plt.show()
print(y_train[:100])

In [ ]:
size = x_train.shape[1]*x_train.shape[2]
x_train = x_train.reshape((x_train.shape[0], size)) / 255
x_test = x_test.reshape((x_test.shape[0], size)) / 255

yc_train, yc_test = to_categorical(y_train), to_categorical(y_test)

x_train = x_train.astype(np.float32)
x_test = x_test.astype(np.float32)
yc_train = yc_train.astype(np.float32)
yc_test = yc_test.astype(np.float32)

In [ ]:
def softmax(z):
    exp = tf.exp(z)
    return exp / tf.expand_dims(tf.math.reduce_sum(exp, axis=1), axis=-1)

def predict(x, w, b):
    return softmax(tf.matmul(x, w) + b)

def categorical_crossentropy(y_true, y_pred):
    return tf.math.reduce_mean(-tf.math.reduce_sum(y_true * tf.math.log(tf.clip_by_value(y_pred, 1e-6, 1)), axis=1))

def loss_f(y_true, x, w, b):
    return categorical_crossentropy(y_true, predict(x, w, b))

In [ ]:
w = tf.random.uniform(shape=[size, 10], minval=-1, maxval=1)
b = tf.random.uniform(shape=[10], minval=-1, maxval=1)
ciclos = 100
lr = 0.01 
momentum = 0.9
mini_batch_size = 500
errors = []
errors_minibatch = []
vw = tf.zeros(shape=[size, 10])
vb = tf.zeros(shape=[10])
for i in tqdm(range(ciclos)):
    x_s, y_s = shuffle(x_train, yc_train)
    for mini_batch in range(0, x_s.shape[0], mini_batch_size):
        with tf.GradientTape() as g:
            g.watch([w, b])
            loss = loss_f(y_s[mini_batch:mini_batch+mini_batch_size], x_s[mini_batch:mini_batch+mini_batch_size], w, b)
            errors_minibatch.append(loss.numpy())
        gw, gb = g.gradient(loss, [w, b])
        vw = momentum * vw - lr * gw
        vb = momentum * vb - lr * gb
        w = w + vw
        b = b + vb
    errors.append(loss_f(yc_train, x_train, w, b).numpy())
    
print('Errores a medida que se actualiza el valor de w')
plt.plot(errors)
plt.show()
plt.plot(errors_minibatch)
plt.show()
print('El w final es {}'.format(w))
print('El b final es {}'.format(b))

In [ ]:
y_pred = predict(x_test, w, b).numpy()
y_pred = np.argmax(y_pred, axis=1)
show_confusion_matrix(confusion_matrix(y_test, y_pred), list(map(str, range(10))))
print(classification_report(y_test, y_pred))

In [ ]:
import seaborn as sn
import pandas as pd
import matplotlib.pyplot as plt
array = confusion_matrix(y_test, y_pred)
df_cm = pd.DataFrame(array, index = [str(i) for i in range(10)],
                  columns = [str(i) for i in range(10)])

sn.heatmap(df_cm, annot=True, fmt="d")
plt.show()

# Optimización con variables

In [ ]:
w = tf.Variable(tf.random.uniform(shape=[size, 10], minval=-1, maxval=1))
b = tf.Variable(tf.random.uniform(shape=[10], minval=-1, maxval=1))
ciclos = 100
lr = 0.01 
momentum = 0.9
mini_batch_size = 500
errors = []
errors_minibatch = []
vw = tf.Variable(tf.zeros(shape=[size, 10]))
vb = tf.Variable(tf.zeros(shape=[10]))
for i in tqdm(range(ciclos)):
    x_s, y_s = shuffle(x_train, yc_train)
    for mini_batch in range(0, x_s.shape[0], mini_batch_size):
        with tf.GradientTape() as g:
            g.watch([w, b])
            loss = loss_f(y_s[mini_batch:mini_batch+mini_batch_size], x_s[mini_batch:mini_batch+mini_batch_size], w, b)
            errors_minibatch.append(loss.numpy())
        gw, gb = g.gradient(loss, [w, b])
        vw.assign(momentum * vw - lr * gw) 
        vb.assign(momentum * vb - lr * gb)
        w.assign(w + vw)
        b.assign(b + vb)
    errors.append(loss_f(yc_train, x_train, w, b).numpy())
    
print('Errores a medida que se actualiza el valor de w')
plt.plot(errors)
plt.show()
plt.plot(errors_minibatch)
plt.show()
print('El w final es {}'.format(w))
print('El b final es {}'.format(b))

In [ ]:
y_pred = predict(x_test, w, b).numpy()
y_pred = np.argmax(y_pred, axis=1)
show_confusion_matrix(confusion_matrix(y_test, y_pred), list(map(str, range(10))))
print(classification_report(y_test, y_pred))

In [ ]:
from tensorflow.keras.layers import Input, Activation
x = np.linspace(-10, 10, 100).astype(np.float32)
for f in ['relu','selu', 'tanh', 'softplus', 'softsign', 'sigmoid', 'tanh', 'softsign']:
    print(f)
    v = Activation(f)(x).numpy()
    plt.ylabel(f)
    plt.plot(x, v)
    plt.show()